In [105]:
import pandas as pd
import math

In [106]:
acs_df = pd.read_csv("Data/ACS_2019_data.csv")
acs_meta_df = pd.read_csv('Data/ACS_2019_meta_data.csv')
demographics_df = pd.read_csv('Data/analytic_data2019.csv').drop(index = 0, columns=['State FIPS Code','County FIPS Code',
                                                                          'State Abbreviation','Release Year'])


/opt/conda/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3165: DtypeWarning: Columns (2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,100,101,102,103,104,105,106,107,108,109,110,111,112,113,114,115,116,117,118,119,120,121,122,123,124,125,126,127,128,129,130,131,132,133,134,135,136,137,138,139,140,141,142,143,144,145,146,147,148,149,150,151,152,153,154,155,156,157,158,159,160,161,162,163,164,165,166,167,168,169,170,171,172,173,174,175,176,177,178,179,180,181,182,183,184,185,186,187,188,189,190,191,192,193,194,195,196,197,198,199,200,201,202,203,204,205,206,207,208,209,210,211,212,213,214,215,216,217,218,219,220,221,222,223,224,225,226,227,228,229,230,231,232,233,234,235,236,237,238,239,240,241,242,243,244,245,246,247,248,249,250,251,252,2

In [107]:
demographics_df = demographics_df.rename(columns={'5-digit FIPS Code': 'county_id'})
demographics_df['county_id'] = demographics_df['county_id'].apply(lambda x: str(x).strip())

In [108]:
# Picking out interested dependent variable column
uninsured_rate_col = 'Estimate!!Percent Uninsured!!Civilian noninstitutionalized population'
id_column = acs_meta_df[acs_meta_df['id']== uninsured_rate_col]['GEO_ID'].loc[489]
rename_dict = {id_column : 'percent_uninsured', 'NAME': 'county_name', 'GEO_ID': 'county_id'}

In [109]:
# Drop the first index, and only pick out three columns to keep before merging redundant info
acs_df = acs_df[['GEO_ID','NAME', id_column]].drop(0)

In [110]:
# Rename the df, and correct the column formatting
acs_df = acs_df.rename(columns=rename_dict)

acs_df['county_name'] = acs_df['county_name'].apply(lambda x: x.strip())
acs_df['state'] = acs_df['county_name'].apply(lambda x: x.split(",")[1].strip())
acs_df['county_id'] = acs_df['county_id'].apply(lambda x: x.split("US")[1].strip())

In [111]:
final_df = acs_df.merge(demographics_df, how='inner', on='county_id')
final_df = final_df.drop(columns=['Name'])
final_df

,county_id,county_name,percent_uninsured,state,County Ranked (Yes=1/No=0),Premature death raw value,Premature death numerator,Premature death denominator,Premature death CI low,Premature death CI high,...,Male population 18-44 raw value,Male population 45-64 raw value,Male population 65+ raw value,Total male population raw value,Female population 0-17 raw value,Female population 18-44 raw value,Female population 45-64 raw value,Female population 65+ raw value,Total female population raw value,Population growth raw value
0,01001,"Autauga County, Alabama",7.1,Alabama,1,8824.0571232,815,156132,7935.3258295,9712.7884169,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,01003,"Baldwin County, Alabama",8.9,Alabama,1,7224.6321603,2827,576496,6794.1283802,7655.1359403,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,01005,"Barbour County, Alabama",11.3,Alabama,1,9586.165037,451,72222,8200.1175409,10972.212533,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,01007,"Bibb County, Alabama",10.7,Alabama,1,11783.543675,445,63653,10159.978321,13407.109028,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,01009,"Blount County, Alabama",10.8,Alabama,1,10908.101822,1050,161107,9895.5825723,11920.621071,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3137,56037,"Sweetwater County, Wyoming",11.3,Wyoming,1.0,7497.439952,495,127427,6599.947576,8394.932328,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3138,56039,"Teton County, Wyoming",12.7,Wyoming,1.0,3786.128226,124,66351,2819.782887,4752.473565,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3139,56041,"Uinta County, Wyoming",11.2,Wyoming,1.0,7790.302043,262,59466,6427.289477,9153.314609,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3140,56043,"Washakie County, Wyoming",15.0,Wyoming,1.0,5504.65097,108,22335,3712.604945,7296.696995,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
# Format all columns to correct data type, rename column names to readable format
for column in final_df.columns:
    # Drop column if all values are NaN
    if final_df[column].isnull().all():
        final_df = final_df.drop(columns=column)
    else:
        if column not in ['county_id', 'state', 'county_name']:
            if column == 'County Ranked (Yes=1/No=0)':
                final_df[column] = final_df[column].apply(lambda x: int(x))
            else:
                final_df[column] = final_df[column].apply(lambda x: float(x))
        # Rename columns
        final_df = final_df.rename(columns = {column: column.lower().replace(" ", "_")})

    # Drop column if 

In [ ]:
final_df.head()

In [ ]:
df.to_csv("Data/Cleaned_Dataset.csv")